# Prepare Files for qpAdm

In [1]:
import numpy as np
import os  # For Saving to Folder
import pandas as pd
import matplotlib.pyplot as plt

import socket
import os as os
import sys as sys
import multiprocessing as mp
import itertools as it
from time import time

# For Arial Font
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'   # Set the defaul
# Make sure to have the font installed (it is on cluster for Harald)
rcParams['font.sans-serif'] = ['Arial']

socket_name = socket.gethostname()
print(socket_name)

if socket_name.startswith("compute-"):
    print("HSM Computational partition detected.")
    path = "/n/groups/reich/hringbauer/git/punic_aDNA/"  # The Path on Midway Cluster
else:
    raise RuntimeWarning("Not compatible machine. Check!!")

os.chdir(path)  # Set the right Path (in line with Atom default)
# Show the current working directory. Should be HAPSBURG/Notebooks/ParallelRuns
print(os.getcwd())
print(f"CPU Count: {mp.cpu_count()}")
print(sys.version)

compute-e-16-229.o2.rc.hms.harvard.edu
HSM Computational partition detected.
/n/groups/reich/hringbauer/git/punic_aDNA
CPU Count: 28
3.7.4 (default, Sep 11 2019, 11:24:51) 
[GCC 6.2.0]


# Key Definitions

In [2]:
base_path = "/n/groups/reich/DAVID/V42/V42.3/v42.3"
ind_path = base_path + ".ind"

In [6]:
#a13_new = ["Sweden_HG_Motala", "Russia_Ust_Ishim.DG", "Russia_Kostenki14.SG",
#          "Goyet_Neanderthal.SG", "Czech_Vestonice16", "Russia_MA1_HG.SG",  "Iberia_ElMiron",
#           "Italy_North_HG_Villabruna", "Russia_HG_Karelia", "Israel_Natufian", "Iran_N_GanjDareh"]
#
#distal = ["Anatolia_N", "Iberia_HG", 
#          "Morocco_Iberomaurusian", 
#          "Russia_EBA_Yamnaya_Samara"]

#print("M13: %i popoulations" % len(m13))
#print("Anc_Euro: %i populations" % len(a13_new))

# Save Population File (what to pull)

In [3]:
def return_pops(df, string, col="clst", 
                output=False):
    """Return list of clusters that contain string."""
    df1 = df[df[col].str.contains(string)]
    if output:
        print(df1[col].value_counts())
    clsts = list(set(df1[col].values))

    return clsts

In [4]:
df_ind = pd.read_csv(ind_path, delim_whitespace=True, header=None)
df_ind.columns=["iid", "sex", "clst"]
print(f"Loaded {len(df_ind)} Individuals")

Loaded 16667 Individuals


In [13]:
##############################################
### Testing for single Populations Populations
return_pops(df_ind, string="Italy_Sardinia_C_o", 
            output=True);

Italy_Sardinia_C_o    1
Name: clst, dtype: int64


In [14]:
pops = ["Algeria", "Morocco", "Punic", "Phoenician", 
        "Canaanite", "Ashkelon", "Greek_Sicily", "Sicily_IA",
        "Israel_IA", "Israel_EIA", "Israel_Persian", "Gibraltar",
        "Iberia_North_BA_Africa_all", "Iberia_BellBeaker_o", "Iberia_Greek",
        "Iberia_Hellenistic", "Nigeria_IA", "Nigeria_Medieval", "YRI", "Italy_Sardinia_C_o"]

exclude_strings = ["_lc", "contam"]

In [15]:
clsts = [return_pops(df_ind, string=pop, 
                     output=False) for pop in pops]

clsts = [inner for ls in clsts for inner in ls]
clsts = list(set(clsts)) # Filter to unique Elements
print(f"Loaded {len(clsts)} Populations")

### Exclude Strings
for ex in exclude_strings:
    clsts = [c for c in clsts if ex not in c]
print(f"After Exclusion {len(clsts)}")

Loaded 71 Populations
After Exclusion 54


# Save List of populations to keep

In [17]:
#keep = np.array(["Anatolia_N", "Iberia_HG"])
keep = np.array(clsts)
path_keep = "./parfiles/keep_pops"
np.savetxt(path_keep, keep, fmt="%s")

### Run convertf (with population list to keep)

In [18]:
def run_convertf(path_convertf = "./o2bin/convertf", parfile = "./explore_ntbk/parfiles/convertf.keep.par"):
    """Runs the Downsampling"""
    ! $path_convertf -p $parfile

In [ ]:
%%time
run_convertf(path_convertf = "/n/groups/reich/hringbauer/o2bin/convertf", 
             parfile = "./parfiles/convertf.keep.par")

parameter file: ./parfiles/convertf.keep.par
BASE: /n/groups/reich/hringbauer
DIR: explore_ntbk/v42/V42.3/v42.3
OUT: git/punic_aDNA/eigenstrat/punic1.v43
genotypename: /n/groups/reich/hringbauer/explore_ntbk/v42/V42.3/v42.3.geno
snpname: /n/groups/reich/hringbauer/explore_ntbk/v42/V42.3/v42.3.snp
indivname: /n/groups/reich/hringbauer/explore_ntbk/v42/V42.3/v42.3.ind
genooutfilename: /n/groups/reich/hringbauer/git/punic_aDNA/eigenstrat/punic1.v43.geno
snpoutfilename: /n/groups/reich/hringbauer/git/punic_aDNA/eigenstrat/punic1.v43.snp
indoutfilename: /n/groups/reich/hringbauer/git/punic_aDNA/eigenstrat/punic1.v43.ind
outputformat: PACKEDANCESTRYMAP
hashcheck: YES
poplistname: /n/groups/reich/hringbauer/git/punic_aDNA/parfiles/keep_pops
## /n/groups/reich/hringbauer/o2bin/convertf version: 5711
read 1073741824 bytes
read 2147483648 bytes
read 3221225472 bytes
read 4294967296 bytes
read 5137965171 bytes
packed geno read OK


## Merge in Lazaridis Ancients with mergeit (with population list to keep)

In [31]:
bin_merge_it = "/n/groups/reich/hringbauer/o2bin/mergeit"
parfile_path = "./parfiles/parMergeAddAnc"

In [32]:
%%time
! $bin_merge_it -p $parfile_path

parameter file: ./parfiles/parMergeAddAnc
BASE: /n/groups/reich/hringbauer/git/punic_aDNA
S1: eigenstrat/punic1.v43
S2: eigenstrat/additional/MinMyc
OUT: eigenstrat/combined/punic1.v43
geno1: /n/groups/reich/hringbauer/git/punic_aDNA/eigenstrat/punic1.v43.geno
snp1: /n/groups/reich/hringbauer/git/punic_aDNA/eigenstrat/punic1.v43.snp
ind1: /n/groups/reich/hringbauer/git/punic_aDNA/eigenstrat/punic1.v43.ind
geno2: /n/groups/reich/hringbauer/git/punic_aDNA/eigenstrat/additional/MinMyc.geno
snp2: /n/groups/reich/hringbauer/git/punic_aDNA/eigenstrat/additional/MinMyc.snp
ind2: /n/groups/reich/hringbauer/git/punic_aDNA/eigenstrat/additional/MinMyc.ind
genooutfilename: /n/groups/reich/hringbauer/git/punic_aDNA/eigenstrat/combined/punic1.v43.geno
snpoutfilename: /n/groups/reich/hringbauer/git/punic_aDNA/eigenstrat/combined/punic1.v43.snp
indoutfilename: /n/groups/reich/hringbauer/git/punic_aDNA/eigenstrat/combined/punic1.v43.ind
docheck: YES
hashcheck: NO
allowdups: YES
packed geno read OK
end

# Prepare Individual File [Stand Alone from here]

In [43]:
ind_merged="./eigenstrat/combined/punic1.v43.ind"          # What .ind to load
df_ind = pd.read_csv(ind_merged, delim_whitespace=True, header=None)
df_ind.columns=["iid", "sex", "clst"]
print(f"Loaded {len(df_ind)} Individuals")

Loaded 642 Individuals


In [44]:
def overwrite_ind_df(df, string, col="clst", 
                     output=False, overwrite="", iids=False):
    """Overwrite Individual Dataframe where col
    contains string. Return modified dataframe (Copy)
    where overwrite is the new Cluster ID
    iids: Overwrite with IIDs if given!"""
    idx = df[col].str.contains(string)
    
    if np.sum(idx)==0:
        if output: 
            print("No Indivdiuals found")
        return
    
    if output:
        print(f"Found {np.sum(idx)} Matches")
        print(df[idx][col].value_counts())
    
    ### Actually  overwrite the Column
    if len(overwrite)>0:
        df.loc[idx, col] = overwrite
        if output: 
            print(f"{np.sum(idx)} Overwritten!")
            
    if iids:
        df.loc[idx, col] = df.loc[idx, "iid"] 

### Test around [don't run]

In [51]:
overwrite_ind_df(df_ind, "Canaanite", 
                 output=True, overwrite="")

#YRI.SG
#Gibraltar_N
#Italy_Sardinia_C_o
#Nigeria_IA

Found 40 Matches
Israel_MLBA_Canaanite                                 26
Lebanon_MLBA_Canaanite_MBA.SG                          5
Israel_MLBA_Canaanite_in.preparation                   2
Israel_MLBA_Canaanite_brother.I10770                   1
Israel_MLBA_Canaanite_brother.I10359                   1
Israel_MLBA_Canaanite_sister.I10101_in.preparation     1
Israel_MLBA_Canaanite_sister.I10101                    1
Israel_MLBA_Canaanite_son.I8187                        1
Israel_MLBA_Canaanite_o                                1
Israel_MLBA_Canaanite_o_brother.I2200                  1
Name: clst, dtype: int64


### Actually overwrite

In [46]:
### Overwrite Group Names
overwrite_ind_df(df_ind, "Morocco_LN", overwrite="Morocco_LN")
overwrite_ind_df(df_ind, "Morocco_EN", overwrite="Morocco_EN")
overwrite_ind_df(df_ind, "Morocco_Iberomaurusian", overwrite="Morocco_Iberomaurusian")
overwrite_ind_df(df_ind, "YRI", overwrite="YRI")

In [49]:
### Overwrite Individual IIDs
overwrite_ind_df(df_ind, "Algeria_N", 
                 iids=True, output=True)
overwrite_ind_df(df_ind, "Punic", 
                 iids=True, output=True)
overwrite_ind_df(df_ind, "Italy_Sicily_Phoenician", 
                 iids=True, output=True);
overwrite_ind_df(df_ind, "Morocco_LN", 
                 iids=True, output=True)
overwrite_ind_df(df_ind, "Punic_oAfrican", 
                 iids=True, output=True)
overwrite_ind_df(df_ind, "Iberia_North_BA_Africa_all", 
                 iids=True, output=True)
overwrite_ind_df(df_ind, "Iberia_BellBeaker_o", 
                 iids=True, output=True)
overwrite_ind_df(df_ind, "Gibraltar_N", 
                 iids=True, output=True)
overwrite_ind_df(df_ind, "Italy_Sardinia_C_o", 
                 iids=True, output=True)
overwrite_ind_df(df_ind, "Nigeria_IA", 
                 iids=True, output=True)
overwrite_ind_df(df_ind, "Nigeria_Medieval", 
                 iids=True, output=True)

No Indivdiuals found
No Indivdiuals found
No Indivdiuals found
No Indivdiuals found
No Indivdiuals found
No Indivdiuals found
No Indivdiuals found
No Indivdiuals found
No Indivdiuals found
No Indivdiuals found
No Indivdiuals found


### Save to Drive here
mod.ind is standard (Morocco_LN not split up)  
mod1.ind is with Morocc_LN split up  

In [48]:
### Save
ind_modified="./eigenstrat/combined/punic1.v43_mod1.ind"    # Where to save the modified version to
df_ind.to_csv(ind_modified, sep=" ", index=None, header=False)
print(f"Saved {len(df_ind)} Individuals to {ind_modified}")

Saved 642 Individuals to ./eigenstrat/combined/punic1.v43_mod1.ind


# Area 51

In [62]:
ind_merged="./eigenstrat/combined/punic1.v43.ind"          # What .ind to load
df_ind = pd.read_csv(ind_merged, delim_whitespace=True, header=None)
df_ind.columns=["iid", "sex", "clst"]
print(f"Loaded {len(df_ind)} Individuals")

Loaded 642 Individuals


In [58]:
overwrite_ind_df(df_ind, "Israel_MLBA_Canaanite", output=True)

Found 35 Matches
Israel_MLBA_Canaanite                                 26
Israel_MLBA_Canaanite_in.preparation                   2
Israel_MLBA_Canaanite_brother.I10770                   1
Israel_MLBA_Canaanite_brother.I10359                   1
Israel_MLBA_Canaanite_sister.I10101_in.preparation     1
Israel_MLBA_Canaanite_sister.I10101                    1
Israel_MLBA_Canaanite_son.I8187                        1
Israel_MLBA_Canaanite_o                                1
Israel_MLBA_Canaanite_o_brother.I2200                  1
Name: clst, dtype: int64


In [67]:
df_ind[df_ind["clst"].str.contains("Iberia_Punic")]

,iid,sex,clst
193,I12517,M,Iberia_Punic
233,I15434,F,Iberia_Punic
244,I18193,M,Iberia_Punic_oAfrican2
245,I18199,M,Iberia_Punic_o.3rd.degree.relative.cluster
246,I18202,M,Iberia_Punic_o.3rd.degree.relative.cluster
247,I18187,F,Iberia_Punic_oEuropean1
248,I18194,M,Iberia_Punic_oAfrican1
249,I18201,M,Iberia_Punic_o.3rd.degree.relative.cluster
250,I18191,F,Iberia_Punic_oEuropean2
251,I18400,F,Iberia_Punic_oAfrican1
